In [0]:
#Initialize PySpark session
from pyspark .sql import SparkSession
from pyspark.sql.functions import col, regexp_replace

#Initialize Spark session
spark = SparkSession.builder.appName("SparkSQL_CreditCard").getOrCreate()

# Step 1: Load the datasets

credit_card_path = "dbfs:/FileStore/tables/credit_card__1_-1.csv"
txn_path = "dbfs:/FileStore/tables/txn__1_-1.csv"

# Load credit card dataset
credit_card_df = spark.read.csv(credit_card_path, header=True, inferSchema=True)
credit_card_df.createOrReplaceTempView("credit_card_data")

# Load txn dataset
txn_df = spark.read.csv(txn_path, header=True, inferSchema=True)
txn_df.createOrReplaceTempView("txn_data")

In [0]:
# step 2: Inspect datasets
print("Credit Card Dataset Schema:")
credit_card_df.printSchema()
credit_card_df.show(5)

print("Transaction Dataset Schema:")
txn_df.printSchema()
txn_df.show(5)

Credit Card Dataset Schema:
root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)

+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|Ha

In [0]:
# Step 3: Filter Transformation

# PySpark
filtered_pyspark = credit_card_df.filter(credit_card_df['CreditScore'] > 800)
filtered_pyspark.show()

# Spark SQL
filtered_sql = spark.sql("SELECT * FROM credit_card_data WHERE CreditScore > 800")
filtered_sql.show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        5|  15737888| Mitchell|        850|    Spain|Female| 43|     2|125510.82|            1|             1|        79084.1|     0|
|        7|  15592531| Bartlett|        822|   France|  Male| 50|     7|      0.0|            2|             1|        10062.8|     0|
|       25|  15625047|      Yen|        846|   France|Female| 38|     5|      0.0|            1|             1|      187616.16|     0|
|       38|  15729599|  Lorenzo|        804|    Spain|  Male| 33|     7|  76548.6|            1|             1|       98453.45|     0|
|       39|  15717426|Armstrong|        850|   France| 

In [0]:
# Step 4: Aggregations

# PySpark
aggregated_pyspark = credit_card_df.groupBy().sum("CreditScore").withColumnRenamed("sum(Amount)", "TotalAmount")
aggregated_pyspark.show()

# Spark SQL
aggregated_sql = spark.sql("SELECT SUM(CreditScore) AS TotalAmount FROM credit_card_data")
aggregated_sql.show()

+----------------+
|sum(CreditScore)|
+----------------+
|         6505288|
+----------------+

+-----------+
|TotalAmount|
+-----------+
|    6505288|
+-----------+



In [0]:
# Step 5 : GroupBy

# PySpark
grouped_pyspark = credit_card_df.groupBy("CreditScore").sum("Balance").withColumnRenamed("sum(Amount)", "TotalAmount")
grouped_pyspark.show()

# Spark SQL
grouped_sql = spark.sql("""
    SELECT Balance, SUM(CreditScore) AS TotalAmount
    FROM credit_card_data
    GROUP BY Balance
""")
grouped_sql.show()

+-----------+------------------+
|CreditScore|      sum(Balance)|
+-----------+------------------+
|        833| 659969.3400000001|
|        496|         893885.63|
|        463|428631.69999999995|
|        471|         474321.42|
|        623|2112271.9600000004|
|        737|1826049.2800000003|
|        540|1889611.9800000002|
|        516|2236659.6300000004|
|        808|         832733.99|
|        580|        3220036.84|
|        451|         202414.29|
|        458|         676461.95|
|        804|447478.44999999995|
|        799|        1254067.02|
|        588|        2382125.17|
|        481| 602554.3300000001|
|        472|         320550.94|
|        513|        1071263.86|
|        673|2516799.1600000006|
|        613|        4266145.49|
+-----------+------------------+
only showing top 20 rows

+---------+-----------+
|  Balance|TotalAmount|
+---------+-----------+
| 128173.9|        639|
| 88109.81|        626|
|  86605.5|        650|
| 97257.41|        750|
| 54817.55|   

In [0]:
# Step 6: Joins

# PySpark Inner Join
inner_join_pyspark = credit_card_df.join(txn_df, credit_card_df["CreditScore"] == txn_df["TRANSACTION DETAILS"], "inner")
inner_join_pyspark.show(5)

# Spark SQL Inner Join
credit_card_df.createOrReplaceTempView("credit_card_data")
txn_df.createOrReplaceTempView("txn_data")

# Perform the join using Spark SQL
inner_join_sql = spark.sql("""
SELECT ccd.CreditScore, txn.`TRANSACTION DETAILS`
FROM credit_card_data ccd
INNER JOIN txn_data txn
ON ccd.CreditScore = txn.`TRANSACTION DETAILS`

""")
inner_join_sql.show(5)

+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+--------------+---------------+------+----------+-------------------+----------+----------------+-------------+-----------+
|RowNumber|CustomerId|Surname|CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Account No|TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+--------------+---------------+------+----------+-------------------+----------+----------------+-------------+-----------+
+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+--------------+---------------+------+----------+-------------------+----------+----------------+-------------+-----------+

+-----------+-------------------+
|CreditScore|TRANSACTION DETAILS|
+-----------+-------------------+
+-----------+-------------------+

In [0]:
# Step 7 : Left join

# Left join in PySpark
left_join_pyspark = credit_card_df.join(txn_df, credit_card_df["CreditScore"] == txn_df["TRANSACTION DETAILS"], "left")
left_join_pyspark.show(5)

# Left join in Spark SQL
left_join_sql = spark.sql("""
SELECT ccd.CreditScore, txn.`TRANSACTION DETAILS`
FROM credit_card_data ccd
LEFT JOIN txn_data txn
ON ccd.CreditScore = txn.`TRANSACTION DETAILS`
""")
left_join_sql.show(5)


+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+----------+-------------------+----------+----------------+-------------+-----------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|Account No|TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+----------+-------------------+----------+----------------+-------------+-----------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|      null|               null|      null|            null|         null|       null|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.5

In [0]:
# Step 8 : Right Join

# Right join in PySpark
right_join_pyspark = credit_card_df.join(txn_df, credit_card_df["CreditScore"] == txn_df["TRANSACTION DETAILS"], "right")
right_join_pyspark.show(5)

# Right join in Spark SQL
right_join_sql = spark.sql("""
SELECT ccd.CreditScore, txn.`TRANSACTION DETAILS`
FROM credit_card_data ccd
RIGHT JOIN txn_data txn
ON ccd.CreditScore = txn.`TRANSACTION DETAILS`
""")
right_join_sql.show(5)



+---------+----------+-------+-----------+---------+------+----+------+-------+-------------+--------------+---------------+------+-------------+--------------------+----------+----------------+-------------+-----------+
|RowNumber|CustomerId|Surname|CreditScore|Geography|Gender| Age|Tenure|Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+---------+----------+-------+-----------+---------+------+----+------+-------+-------------+--------------+---------------+------+-------------+--------------------+----------+----------------+-------------+-----------+
|     null|      null|   null|       null|     null|  null|null|  null|   null|         null|          null|           null|  null|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|            null|    1000000.0|  1000000.0|
|     null|      null|   null|       null|     null|  null|null|  null|   null|         null|          null|        

In [0]:
# Step 9 : Outer Join

# Outer join in PySpark
outer_join_pyspark = credit_card_df.join(txn_df, credit_card_df["CreditScore"] == txn_df["TRANSACTION DETAILS"], "outer")
outer_join_pyspark.show(5)

# Outer join in Spark SQL
outer_join_sql = spark.sql("""
SELECT ccd.CreditScore, txn.`TRANSACTION DETAILS`
FROM credit_card_data ccd
FULL OUTER JOIN txn_data txn
ON ccd.CreditScore = txn.`TRANSACTION DETAILS`
""")
outer_join_sql.show(5)


+---------+----------+-------+-----------+---------+------+----+------+-------+-------------+--------------+---------------+------+-------------+--------------------+----------+----------------+-------------+-----------+
|RowNumber|CustomerId|Surname|CreditScore|Geography|Gender| Age|Tenure|Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+---------+----------+-------+-----------+---------+------+----+------+-------+-------------+--------------+---------------+------+-------------+--------------------+----------+----------------+-------------+-----------+
|     null|      null|   null|       null|     null|  null|null|  null|   null|         null|          null|           null|  null|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|            null|    1000000.0|  1000000.0|
|     null|      null|   null|       null|     null|  null|null|  null|   null|         null|          null|        

In [0]:
# Step 10 : Cross Join

# Perform a cross join in PySpark
cross_join_pyspark = credit_card_df.crossJoin(txn_df)
cross_join_pyspark.show(5)

# Register the DataFrames as SQL temporary views
credit_card_df.createOrReplaceTempView("credit_card_data")
txn_df.createOrReplaceTempView("txn_data")

# Perform a cross join in Spark SQL
cross_join_sql = spark.sql("""
SELECT *
FROM credit_card_data ccd
CROSS JOIN txn_data txn
""")
cross_join_sql.show(5)


+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+-------------+--------------------+----------+----------------+-------------+-----------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+-------------+--------------------+----------+----------------+-------------+-----------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|            null|    1000000.0|  1000000.0|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             